Some simple scripts to manage datas

In [1]:
import polars as pl

In [ ]:
# Votes per measure
data = pl.read_csv("assets/evaluation/data_kano.csv", separator=";")

measure_count = data.get_column("mesure").value_counts()

measure_count.write_csv("assets/evaluation/num_votes_per_measure.csv")

print(measure_count)

shape: (12, 2)
┌────────┬───────┐
│ mesure ┆ count │
│ ---    ┆ ---   │
│ str    ┆ u32   │
╞════════╪═══════╡
│ m13    ┆ 9     │
│ m5     ┆ 9     │
│ m1     ┆ 9     │
│ m17    ┆ 7     │
│ m3     ┆ 7     │
│ …      ┆ …     │
│ m2     ┆ 6     │
│ m6     ┆ 9     │
│ m4     ┆ 9     │
│ m10    ┆ 9     │
│ m9     ┆ 9     │
└────────┴───────┘
